In [ ]:

# 🧠 Clasificador de Sentimiento - Proyecto API Hackathon

## 1. Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

# Configuraciones visuales
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 5)

## 2. Cargar dataset
df = pd.read_csv("dataset_sentimientos_ecom.csv")

## 3. Limpieza del texto
# Lista básica de stopwords en español
stopwords_es = set("""
de la que el en y a los del se las por un para con no una su al lo como más pero sus le ya o este sí porque esta entre cuando muy sin sobre también me hasta hay donde quien desde todo nos durante todos uno les ni contra otros ese eso ante ellos e esto mí antes algunos qué unos yo otro otras otra él tanto esa estos mucho quienes nada muchos cual poco ella estar estas algunas algo nosotros mi mis tú te ti tu tus ellas nosotras vosotras vosotros os mío mía míos mías tuyo tuya tuyos tuyas suyo suya suyos suyas nuestro nuestra nuestros nuestras vuestro vuestra vuestros vuestras esos esas estoy estás está estamos estáis están esté estés estemos estéis estén estaré estarás estará estaremos estaréis estarán estaría estarías estaríamos estaríais estarían estaba estabas estábamos estabais estaban estuve estuviste estuvo estuvimos estuvisteis estuvieron estuviera estuvieras estuviéramos estuvierais estuvieran estuviese estuvieses estuviésemos estuvieseis estuviesen estando estado estada estados estadas estad
""".split())

# Función de limpieza
def limpieza_basica(texto):
    texto = texto.lower()
    texto = re.sub(r"[^\w\s]", "", texto)
    texto = re.sub(r"\d+", "", texto)
    palabras = texto.split()
    palabras = [p for p in palabras if p not in stopwords_es]
    return " ".join(palabras)

df['texto_limpio'] = df['texto'].apply(limpieza_basica)

# Eliminar textos con menos de 5 palabras
df = df[df['texto_limpio'].apply(lambda x: len(x.split()) >= 5)].reset_index(drop=True)

## 4. Visualización distribución de clases
sns.countplot(data=df, x='sentimiento')
plt.title("Distribución de sentimientos")
plt.show()

## 5. Vectorización TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['texto_limpio'])
y = df['sentimiento']

## 6. División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

## 7. Entrenamiento del modelo
modelo = LogisticRegression(max_iter=1000)
modelo.fit(X_train, y_train)

## 8. Evaluación
y_pred = modelo.predict(X_test)
print(classification_report(y_test, y_pred))

## 9. Serialización
joblib.dump(modelo, "modelo_sentimiento.joblib")
joblib.dump(vectorizer, "tfidf_vectorizer.joblib")
